In [ ]:
import urllib.request as url
from bs4 import BeautifulSoup
import requests
import csv
import math

In [ ]:
def get_review_id(business_id, business_review, review_date):
    business_review = business_review.replace(" ","")
    review_id = ''
    if len(business_review) > 10:
        review_id = business_id + (business_review[0:9]) + str(review_date)
    else:
        num_chars = 10 - len(business_review)
        review_id = business_id + business_review + 'x' * num_chars + str(review_date)
        
    return review_id

In [ ]:
def write_to_csv(data):
    with open(r'yelp_reviews_data.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(data) 

In [ ]:
def get_review_data_version_two(url):
    
    div_main="review-content"
    div_above_stars="biz-rating__stars"
    div_date="rating-qualifier"
    
    one_star_div = 'i-stars i-stars--regular-1 rating-large'
    two_star_div = 'i-stars i-stars--regular-2 rating-large'
    three_star_div = 'i-stars i-stars--regular-3 rating-large'
    four_star_div = 'i-stars i-stars--regular-4 rating-large'
    five_star_div = 'i-stars i-stars--regular-5 rating-large'
    
    url = row[0]
    iterations = math.ceil(int(row[2]) / 20)
    
    rating = ''
    review = ''
    review_id = ''
    
    counter = 0
    web_page_incrementor = 0
           
    while True:
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for divs in soup.find_all(class_=div_main):  
            
            row_lst = []
            
            review_date = (str(divs.find(class_=div_date).text).lstrip())[0:9].strip()
            
            if '/2019' in review_date:
            
                if (divs.find(class_=five_star_div))!= None:
                    rating = 5

                elif (divs.find(class_=four_star_div)) != None:
                    rating = 4

                elif (divs.find(class_=three_star_div)) != None:
                    rating = 3

                elif (divs.find(class_=two_star_div)) != None:
                    rating = 2

                else:
                    rating = 1

                review = divs.find('p').text
                review_id = get_review_id(row[1], review, review_date)
                
                row_lst.append(review_id)
                row_lst.append(row[1])
                row_lst.append(rating)
                row_lst.append(review_date)
                row_lst.append(divs.find('p').text)
                
                write_to_csv(row_lst)
            
                counter += 1
                print('row written using Version 2')
        
        iterations -= 1
        
        if counter == 0 or iterations == 0:
            break
        counter = 0
        web_page_incrementor += 20
        add_on_url = '&start=' + str(web_page_incrementor)
        url = row[0] + add_on_url

In [ ]:
def get_review_data_version_one(row):
    
    main_div = "lemon--div__373c0__1mboc sidebarActionsHoverTarget__373c0__2kfhE arrange__373c0__UHqhV gutter-12__373c0__3kguh layout-stack-small__373c0__3cHex border-color--default__373c0__2oFDT"
    review_div = "lemon--p__373c0__3Qnnj text__373c0__2pB8f comment__373c0__3EKjH text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"
    stars_div = "lemon--div__373c0__1mboc i-stars__373c0__3UQrn i-stars--regular-5__373c0__3kPgP border-color--default__373c0__2oFDT overflow--hidden__373c0__8Jq2I"     
    date_div = "lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"   
    div_above_star="lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT"
       
    url = row[0]
    iterations = math.ceil(int(row[2]) / 20)
    web_page_incrementor = 0
    counter = 0
    
    rating = ''
    review = ''
    review_id = ''
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')                
        
        for divs in soup.find_all(class_=main_div):
            
            row_lst = []
            review_date = review_date = divs.find(class_=date_div).text
            
            if '/2019' in review_date:
                           
                for s in (divs.find(class_=div_above_star)):

                    rating = int(s['aria-label'][0])

                review = divs.find(class_=review_div).text

                row_lst.append(get_review_id(row[1], review, review_date))
                row_lst.append(row[1])
                row_lst.append(rating)
                row_lst.append(review_date)
                row_lst.append(review)

                write_to_csv(row_lst)
     
                counter += 1
                print('row written using Version 1')  
        iterations -= 1
        
        if counter == 0 or iterations == 0:
            break
            
        counter = 0
        web_page_incrementor += 20
        
        add_on_url = '&start=' + str(web_page_incrementor)
        url = row[0] + add_on_url

In [ ]:
business_counter = 0
header = True
with open('for_scraping.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if not header:
            response = requests.get(row[0])
            soup = BeautifulSoup(response.text, 'html.parser')
        
            if (soup.find_all(class_="review-content")) == []:
                get_review_data_version_one(row)
            else:
                get_review_data_version_two(row)
            
            business_counter += 1
            
            print(str(business_counter) + '. business is done.')
        
        else: 
            header = False